In [1]:
import pandas as pd
import openpyxl
import time
import re
#兩張表：A到I欄位相同，「sheet1」為原始表格，「工作表1為目標欄位」

In [2]:
df = pd.read_excel("爬蟲需求-價格比價.xlsx",sheet_name=0)

In [3]:
df['賣場名稱'] #檢查資料

0                  ViewSonic VA708a 17型 節能電腦螢幕
1     ViewSonic VX2476-smhd 24型 AH-IPS 薄邊框電腦螢幕
2                  ViewSonic VA2205-H 22型VA寬螢幕
3           ViewSonic VG2448 24型 FHD 窄邊框IPS寬螢幕
4                   ViewSonic VA1903A 19型寬電腦螢幕
5                 MSI微星 Optix G241VC 24型曲面電競螢幕
6                        AOC 27B2H 27吋窄邊框廣視角螢幕
7        ViewSonic VG2755-2K 27型 人體工學設計多角度旋轉螢幕
8                 Samsung U32R591CWC 32型4K曲面螢幕
9                 Samsung C27F390FHE 27型VA曲面螢幕
10                Samsung C24F390FHE 24型VA曲面螢幕
11    ViewSonic VX2776-smhd 27型 AH-IPS 薄邊框電腦螢幕
12                  PHILIPS 325M8C 32型2K曲面電競螢幕
13                   PHILIPS 221V8A 22型 FHD寬螢幕
14               PHILIPS 242V8A 24型 IPS FHD寬螢幕
15             SAMSUNG C27T550FDC 27型1000R曲面螢幕
16             SAMSUNG C32T550FDC 32型1000R曲面螢幕
17                       HP 24mh 24型 IPS 微邊框螢幕
Name: 賣場名稱, dtype: object

In [4]:
#考慮到會用在momo以及pchome進行搜尋，

#可以應用到的為「子站、父子類、供應商名稱、賣場名稱」，主要用到「賣場名稱」

In [5]:
'''
for i in df['賣場名稱']:
    print(i)
'''
df

,賣場編號,商品編號,子站,父子類,供應商名稱,賣場名稱,廠商建議價,賣場價格,商品成本,momo賣場名稱,momo售價,momo連結,PChome賣場名稱,PChome售價,PChome連結
0,6500964,17435585,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (14793) 19型以下螢幕,(548) 建達,ViewSonic VA708a 17型 節能電腦螢幕,4100,4100,xxxx,【ViewSonic 優派】VA708a 17型 5:4 LED節能顯示器,4100.0,https://www.momoshop.com.tw/goods/GoodsDetail....,ViewSonic VA708a 17吋5:4寬螢幕,4100.0,https://24h.pchome.com.tw/prod/DSABEX-A900FHMN...
1,6943083,18789738,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (69718) 24型螢幕,(548) 建達,ViewSonic VX2476-smhd 24型 AH-IPS 薄邊框電腦螢幕,4788,4788,xxxx,NaN,NaN,NaN,NaN,NaN,NaN
2,8064268,22312101,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (22030) 22型螢幕,(548) 建達,ViewSonic VA2205-H 22型VA寬螢幕,2777,2777,xxxx,NaN,NaN,NaN,NaN,NaN,NaN
3,8271999,22769641,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (69718) 24型螢幕,(548) 建達,ViewSonic VG2448 24型 FHD 窄邊框IPS寬螢幕,4688,4688,xxxx,NaN,NaN,NaN,NaN,NaN,NaN
4,8370002,23309081,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (14793) 19型以下螢幕,(548) 建達,ViewSonic VA1903A 19型寬電腦螢幕,2488,2488,xxxx,NaN,NaN,NaN,NaN,NaN,NaN
5,8421422,23477162,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (69718) 24型螢幕,(548) 建達,MSI微星 Optix G241VC 24型曲面電競螢幕,4990,3690,xxxx,NaN,NaN,NaN,NaN,NaN,NaN
6,8710333,24490478,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (97039) 27型螢幕,(548) 建達,AOC 27B2H 27吋窄邊框廣視角螢幕,3788,3788,xxxx,NaN,NaN,NaN,NaN,NaN,NaN
7,8774049,24707118,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (97039) 27型螢幕,(548) 建達,ViewSonic VG2755-2K 27型 人體工學設計多角度旋轉螢幕,10988,10988,xxxx,NaN,NaN,NaN,NaN,NaN,NaN
8,8779685,24724556,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (109824) 32型螢幕,(548) 建達,Samsung U32R591CWC 32型4K曲面螢幕,15900,12888,xxxx,NaN,NaN,NaN,NaN,NaN,NaN
9,8779717,24724778,(22) LCD液晶螢幕,(2149) 電腦螢幕尺寸 - (97039) 27型螢幕,(548) 建達,Samsung C27F390FHE 27型VA曲面螢幕,5990,5990,xxxx,NaN,NaN,NaN,NaN,NaN,NaN


### 程式設計點1：名稱各不相同

觀察第一份商品

*賣場名稱：「ViewSonic VA708a 17型 節能電腦螢幕」*

*MOMO賣場名稱：「【ViewSonic 優派】VA708a 17型 5:4 LED節能顯示器」*

*PCHOME賣場名稱：「ViewSonic VA708a 17吋5:4寬螢幕」*

由此可得知共同點為商品序號「VA708a」以及尺吋「17型(吋)」

此份Excel檔主要類型為「LCD液晶螢幕」，可以針對名稱做正則表達式(針對名稱的方法會在導入不同類型的商品時失效。)